## Feature Engineering
In the last notebook date was seperated to year, month, and day and time was seperated to hour and minute. Now city and county will be added using geopandas.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [2]:
clean_collisions = pd.read_csv('clean_collisions.csv',low_memory=False)

In [3]:
clean_collisions = clean_collisions.drop('Unnamed: 0', axis = 1)

This was the original line of code that set the stage for all the changes that came later in this notebook. It has been made into a comment because the shapefiles used have since been deleted.
```
gdf = gpd.GeoDataFrame(clean_collisions, geometry=gpd.points_from_xy(clean_collisions.LONGITUDE, clean_collisions.LATITUDE))

# load in shapefiles for city and county
county_boundaries = gpd.read_file('CA_Counties_TIGER2016.shp')
city_boundaries = gpd.read_file('City_Boundaries.shp')

# make boundries the same
gdf = gdf.set_crs(county_boundaries.crs, allow_override=True)
city_boundaries = city_boundaries.to_crs(county_boundaries.crs)

# join boundry data with clean_collisions
county_joined = gpd.sjoin(gdf, county_boundaries, how='left', predicate='within')

city_joined = gpd.sjoin(gdf, city_boundaries, how='left', predicate='within')
```

In [4]:
clean_collisions.head(5)

,CASE_ID,AT_FAULT,PARTY_SEX,PARTY_AGE,PARTY_SOBRIETY,PARTY_DRUG_PHYSICAL,DIR_OF_TRAVEL,PARTY_SAFETY_EQUIP_1,PARTY_SAFETY_EQUIP_2,FINAN_RESPONS,...,LIGHTING,CONTROL_DEVICE,ALCOHOL_INVOLVED,LATITUDE,LONGITUDE,COLLISION_YEAR,COLLISION_MONTH,COLLISION_DAY,COLLISION_HOUR,COLLISION_MINUTE
0,6698645,Y,M,31,B,Not Stated,N,M,G,N,...,C,A,Y,34.57264,118.04491,2018,9,30,19,45
1,6698645,N,M,44,A,Not Stated,N,M,G,Y,...,C,A,Y,34.57264,118.04491,2018,9,30,19,45
2,8008483,Y,M,27,B,Not Stated,W,M,G,N,...,C,A,Y,34.21760,119.18680,2020,10,5,22,36
3,8008483,N,Not Stated,42,H,H,W,Not Stated,Not Stated,O,...,C,A,Y,34.21760,119.18680,2020,10,5,22,36
4,8008483,N,Not Stated,42,H,H,W,Not Stated,Not Stated,O,...,C,A,Y,34.21760,119.18680,2020,10,5,22,36


The code above was producing a lot all Nan values for city and county names. After some further analysis I found that the values for longitude have to be negative! So, in the next lines, we will change the data for longitude to negatives. Also, the city shapefile also contains the county so only the city shaprfile will be used.

In [5]:
clean_collisions['LONGITUDE'] = clean_collisions['LONGITUDE'].apply(lambda x: -x if x > 0 else x)

In [6]:
# edited from the code above
gdf = gpd.GeoDataFrame(clean_collisions, geometry=gpd.points_from_xy(clean_collisions.LONGITUDE, clean_collisions.LATITUDE))

gdf = gdf.set_crs("EPSG:4326")

# laod in shapefiles for city and county

city_boundaries = gpd.read_file('City_Boundaries.shp')
city_boundaries = city_boundaries.to_crs("EPSG:4326")

# join boundry data with clean_collisions

city_joined = gpd.sjoin(gdf, city_boundaries, how='left', predicate='within')

In [7]:
# add county and city columns to city_joined clean_collisions

clean_collisions['CITY'] = city_joined['CITY']
clean_collisions['COUNTY'] = city_joined['COUNTY']

In [8]:
clean_collisions[clean_collisions['CITY'].isna()].head(5)

,CASE_ID,AT_FAULT,PARTY_SEX,PARTY_AGE,PARTY_SOBRIETY,PARTY_DRUG_PHYSICAL,DIR_OF_TRAVEL,PARTY_SAFETY_EQUIP_1,PARTY_SAFETY_EQUIP_2,FINAN_RESPONS,...,ALCOHOL_INVOLVED,LATITUDE,LONGITUDE,COLLISION_YEAR,COLLISION_MONTH,COLLISION_DAY,COLLISION_HOUR,COLLISION_MINUTE,CITY,COUNTY
7,8008494,Y,M,47,A,Not Stated,E,M,G,Y,...,N,37.98705,-122.31694,2018,9,5,11,30,NaN,NaN
8,8008494,N,M,31,A,Not Stated,W,L,B,Y,...,N,37.98705,-122.31694,2018,9,5,11,30,NaN,NaN
48,80961973,Y,F,18,A,Not Stated,E,M,G,Y,...,N,35.34322,-119.02190,2019,3,28,80,4,NaN,NaN
49,80961973,N,M,40,A,Not Stated,S,L,G,Y,...,N,35.34322,-119.02190,2019,3,28,80,4,NaN,NaN
205,80983277,Y,F,36,A,Not Stated,W,M,G,Y,...,N,35.35398,-119.07423,2019,5,6,82,2,NaN,NaN


Missing City and County names could be because the lat and lon fall on a boundry. For each of those lat and lon values, the accuracy will be decresed slightly to move it away from a boundry.

In [9]:
# drop city and county columns
clean_collisions = clean_collisions.drop(['CITY','COUNTY'], axis = 1)

In [10]:
# edited from the code above
gdf_2 = gpd.GeoDataFrame(clean_collisions, geometry=gpd.points_from_xy(clean_collisions.LONGITUDE, clean_collisions.LATITUDE))

gdf_2 = gdf_2.set_crs("EPSG:4326")

# laod in shapefiles for city and county

city_boundaries_new = gpd.read_file('City_Boundaries.shp')
city_boundaries_new = city_boundaries_new.to_crs("EPSG:4326")

# join boundry data with clean_collisions

city_joined_new = gpd.sjoin(gdf_2, city_boundaries_new, how='left', predicate='within')

In [11]:
# add county and city columns to city_joined clean_collisions

clean_collisions['CITY'] = city_joined['CITY']
clean_collisions['COUNTY'] = city_joined['COUNTY']

In [12]:
clean_collisions[clean_collisions['CITY'].isna()].head(5)

,CASE_ID,AT_FAULT,PARTY_SEX,PARTY_AGE,PARTY_SOBRIETY,PARTY_DRUG_PHYSICAL,DIR_OF_TRAVEL,PARTY_SAFETY_EQUIP_1,PARTY_SAFETY_EQUIP_2,FINAN_RESPONS,...,ALCOHOL_INVOLVED,LATITUDE,LONGITUDE,COLLISION_YEAR,COLLISION_MONTH,COLLISION_DAY,COLLISION_HOUR,COLLISION_MINUTE,CITY,COUNTY
7,8008494,Y,M,47,A,Not Stated,E,M,G,Y,...,N,37.98705,-122.31694,2018,9,5,11,30,NaN,NaN
8,8008494,N,M,31,A,Not Stated,W,L,B,Y,...,N,37.98705,-122.31694,2018,9,5,11,30,NaN,NaN
48,80961973,Y,F,18,A,Not Stated,E,M,G,Y,...,N,35.34322,-119.02190,2019,3,28,80,4,NaN,NaN
49,80961973,N,M,40,A,Not Stated,S,L,G,Y,...,N,35.34322,-119.02190,2019,3,28,80,4,NaN,NaN
205,80983277,Y,F,36,A,Not Stated,W,M,G,Y,...,N,35.35398,-119.07423,2019,5,6,82,2,NaN,NaN


In [13]:
# attempt with another from data.gov shapefile

# drop city and county columns
clean_collisions = clean_collisions.drop(['CITY','COUNTY'], axis = 1)

gdf_3 = gpd.GeoDataFrame(clean_collisions, geometry=gpd.points_from_xy(clean_collisions.LONGITUDE, clean_collisions.LATITUDE))

gdf_3 = gdf_3.set_crs("EPSG:4326")

# laod in shapefiles for city and county

tl_boundaries = gpd.read_file('tl_2016_06_place.shp')
tl_boundaries = tl_boundaries.to_crs("EPSG:4326")

# join boundry data with clean_collisions

tl_joined_ = gpd.sjoin(gdf_3, tl_boundaries, how='left', predicate='within')

In [14]:
# add city columns to clean_collisions

clean_collisions['CITY'] = tl_joined_['NAME']

In [15]:
# attempt getting county names with another shapefile from ArcGIS

gdf_4 = gpd.GeoDataFrame(clean_collisions, geometry=gpd.points_from_xy(clean_collisions.LONGITUDE, clean_collisions.LATITUDE))

gdf_4 = gdf_4.set_crs("EPSG:4326")

# laod in shapefiles for city and county

county_gis = gpd.read_file('cnty19_1_basicplus.shp')

# join boundry data with clean_collisions

county_gis_joined = gpd.sjoin(gdf_4, county_gis, how='left', predicate='within')

In [16]:
# add county columns to clean_collisions

clean_collisions['COUNTY'] = county_gis_joined['COUNTY_NAM']

In [17]:
clean_collisions[['COUNTY','CITY']].isna().sum()

COUNTY     18274
CITY      619612
dtype: int64

In [18]:
# view the rows that have a city but no county
city_no_county = clean_collisions[clean_collisions['COUNTY'].isna() & clean_collisions['CITY'].notna()]

city_no_county['CITY'].unique()

array(['Newport Beach', 'Huntington Beach', 'South Lake Tahoe', 'Oxnard',
       'Santa Barbara', 'Oceanside', 'Malibu', 'Port Hueneme', 'Pacifica',
       'Los Angeles', 'San Francisco', 'Dana Point', 'Sand City',
       'Coronado', 'Topanga', 'Long Beach', 'San Diego', 'Moss Beach',
       'Monterey', 'San Buenaventura (Ventura)', 'Encinitas',
       'Twin Lakes', 'Rio del Mar', 'Bertsch-Oceanview'], dtype=object)

A lot of coordinates end up in the pacific ocean but get captured in some shapefiles for city but not county. These missing county names will be manually imputed using a dictionary.

In [19]:

# county names aquired from AcrGIS map
county_dict = {
    'Newport Beach': 'Orange',
    'Huntington Beach': 'Orange',
    'South Lake Tahoe': 'El Dorado',
    'Oxnard': 'Ventura',
    'Santa Barbara': 'Santa Barbara',
    'Oceanside': 'San Diego',
    'Malibu': 'Los Angeles',
    'Port Hueneme': 'Ventura',
    'Pacifica': 'San Mateo',
    'Los Angeles': 'Los Angeles',
    'San Francisco': 'San Francisco',
    'Dana Point': 'Orange',
    'Sand City': 'Monterey',
    'Coronado': 'San Diego',
    'Topanga': 'Los Angeles',
    'Long Beach': 'Los Angeles',
    'San Diego': 'San Diego',
    'Moss Beach': 'San Mateo',
    'Monterey': 'Monterey',
    'San Buenaventura (Ventura)': 'Ventura',
    'Encinitas': 'San Diego',
    'Twin Lakes': 'Santa Cruz', 
    'Rio del Mar': 'Santa Cruz',
    'Bertsch-Oceanview': 'Del Norte'
}

# add missing county names if they have a city
county_dict_series = clean_collisions['CITY'].map(county_dict)

# fill in missing values in county with the mapped values from county_dict_series
clean_collisions['COUNTY'] = clean_collisions['COUNTY'].fillna(county_dict_series)

In [20]:
county_no_city = clean_collisions[clean_collisions['CITY'].isna() & clean_collisions['COUNTY'].notna()]

county_no_city.head(5)

,CASE_ID,AT_FAULT,PARTY_SEX,PARTY_AGE,PARTY_SOBRIETY,PARTY_DRUG_PHYSICAL,DIR_OF_TRAVEL,PARTY_SAFETY_EQUIP_1,PARTY_SAFETY_EQUIP_2,FINAN_RESPONS,...,ALCOHOL_INVOLVED,LATITUDE,LONGITUDE,COLLISION_YEAR,COLLISION_MONTH,COLLISION_DAY,COLLISION_HOUR,COLLISION_MINUTE,CITY,COUNTY
9,8008503,Y,F,25,A,Not Stated,S,L,G,N,...,N,34.24107,-119.18772,2020,10,1,18,0,NaN,Ventura
10,8008503,N,F,22,A,Not Stated,N,M,G,Y,...,N,34.24107,-119.18772,2020,10,1,18,0,NaN,Ventura
48,80961973,Y,F,18,A,Not Stated,E,M,G,Y,...,N,35.34322,-119.02190,2019,3,28,80,4,NaN,Kern
49,80961973,N,M,40,A,Not Stated,S,L,G,Y,...,N,35.34322,-119.02190,2019,3,28,80,4,NaN,Kern
200,80983274,Y,F,83,A,Not Stated,E,M,G,Y,...,N,35.33952,-119.02892,2019,5,5,13,47,NaN,Kern


In [21]:
no_county_no_city = clean_collisions[clean_collisions['CITY'].isna() & clean_collisions['COUNTY'].isna()]

no_county_no_city.head(5)

,CASE_ID,AT_FAULT,PARTY_SEX,PARTY_AGE,PARTY_SOBRIETY,PARTY_DRUG_PHYSICAL,DIR_OF_TRAVEL,PARTY_SAFETY_EQUIP_1,PARTY_SAFETY_EQUIP_2,FINAN_RESPONS,...,ALCOHOL_INVOLVED,LATITUDE,LONGITUDE,COLLISION_YEAR,COLLISION_MONTH,COLLISION_DAY,COLLISION_HOUR,COLLISION_MINUTE,CITY,COUNTY
5243,81165343,Y,Not Stated,42,G,G,N,M,B,N,...,N,32.6298,-117.8859,2020,1,7,22,58,NaN,NaN
9381,81253850,N,M,23,A,Not Stated,N,M,G,Y,...,N,33.3742,-117.5708,2020,5,30,13,50,NaN,NaN
9434,81254602,Y,M,57,A,Not Stated,W,Not Stated,W,O,...,N,33.2991,-117.8718,2020,6,10,17,36,NaN,NaN
9435,81254602,N,M,24,A,Not Stated,E,M,G,Y,...,N,33.2991,-117.8718,2020,6,10,17,36,NaN,NaN
12636,81322860,Y,M,33,B,Not Stated,S,M,G,N,...,Y,33.3635,-117.5541,2020,10,1,15,8,NaN,NaN


The remaining missing data for city and country will be filled using the nearest city and county to other nearby coordinates. A lot of points are just outside of the coastline or in a rural area that does not have an incorperated city. 

In [23]:
from sklearn.neighbors import BallTree

gdf_5 = gpd.GeoDataFrame(clean_collisions, geometry=gpd.points_from_xy(clean_collisions.LONGITUDE, clean_collisions.LATITUDE))

# separate rows of known and unknown city/county
known_gdf = gdf_5.dropna(subset=['CITY', 'COUNTY']).copy()
unknown_gdf = gdf_5[gdf_5['CITY'].isna() | gdf_5['COUNTY'].isna()].copy()

# convert latitude and longitude to radians for use in BallTree
known_coords = np.radians(known_gdf[['LATITUDE', 'LONGITUDE']].values)
unknown_coords = np.radians(unknown_gdf[['LATITUDE', 'LONGITUDE']].values)

# create the BallTree with known coordinates
tree = BallTree(known_coords, metric='haversine')

# query the tree for each unknown coordinate to find the nearest known coordinate
dist, ind = tree.query(unknown_coords, k=1)

# impute missing city and county values
# convert 'ind' to a pandas Series to use with 'apply'
indices_series = pd.Series(ind.squeeze())

# use indices_series to map indices to city and county names
unknown_gdf['CITY'] = indices_series.apply(lambda x: known_gdf.iloc[x]['CITY']).values
unknown_gdf['COUNTY'] = indices_series.apply(lambda x: known_gdf.iloc[x]['COUNTY']).values

# add the imputed values back into clean_collisions
for idx in unknown_gdf.index:
    clean_collisions.loc[idx, 'CITY'] = unknown_gdf.loc[idx, 'CITY']
    clean_collisions.loc[idx, 'COUNTY'] = unknown_gdf.loc[idx, 'COUNTY']

In [25]:
clean_collisions.loc[indices_series].sample(5)

,CASE_ID,AT_FAULT,PARTY_SEX,PARTY_AGE,PARTY_SOBRIETY,PARTY_DRUG_PHYSICAL,DIR_OF_TRAVEL,PARTY_SAFETY_EQUIP_1,PARTY_SAFETY_EQUIP_2,FINAN_RESPONS,...,ALCOHOL_INVOLVED,LATITUDE,LONGITUDE,COLLISION_YEAR,COLLISION_MONTH,COLLISION_DAY,COLLISION_HOUR,COLLISION_MINUTE,CITY,COUNTY
741940,90787584,N,F,40,A,Not Stated,E,M,G,Y,...,N,34.06561,-117.49438,2018,8,2,15,25,Fontana,San Bernardino
650499,90735293,N,M,34,A,Not Stated,E,L,G,Y,...,N,33.19407,-117.15178,2018,5,21,52,5,San Marcos,San Diego
1346888,91139013,N,M,28,A,Not Stated,E,M,G,Y,...,N,33.92680,-116.97396,2019,11,29,14,0,Beaumont,Riverside
1413932,91179225,N,F,43,A,Not Stated,N,M,G,Y,...,N,34.09327,-118.24142,2020,1,25,14,35,Los Angeles,Los Angeles
342012,8749698,Y,M,28,A,Not Stated,S,M,G,Not Stated,...,N,33.50450,-117.59420,2018,11,20,21,24,San Juan Capistrano,Orange
